![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XlmRoBertaForSequenceClassification.ipynb)

## Import ONNX XlmRoBertaForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `XlmRoBertaForSequenceClassification` is only available since in `Spark NLP 5.2.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLM-RoBERTa models trained/fine-tuned for sequence classification via `XlmRoBertaForSequenceClassification` or `TFXlmRoBertaForSequenceClassification`. These models are usually under `Text Classification` category and have `xlm-roberta` in their labels
- Reference: [TFXlmRoBertaForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/xlm-roberta#transformers.TFXLMRobertaForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=xlm-roberta&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- Hugging Face has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [cardiffnlp/twitter-xlm-roberta-base-sentiment](https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [5]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

('onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment/tokenizer_config.json',
 'onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment/special_tokens_map.json',
 'onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment/sentencepiece.bpe.model',
 'onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment/added_tokens.json')

Let's have a look inside this and see what we are dealing with:

In [6]:
!ls -l {ONNX_MODEL}

total 1091400
-rw-r--r-- 1 root root        847 Jun 17 05:15 config.json
-rw-r--r-- 1 root root 1112503258 Jun 17 05:15 model.onnx
-rw-r--r-- 1 root root    5069051 Jun 17 05:16 sentencepiece.bpe.model
-rw-r--r-- 1 root root        280 Jun 17 05:16 special_tokens_map.json
-rw-r--r-- 1 root root       1231 Jun 17 05:16 tokenizer_config.json


- As you can see, we need to move `sentencepiece.bpe.model` from the tokenizer to `assets` folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [7]:
from transformers import AutoConfig

!mkdir -p {ONNX_MODEL}/assets

config = AutoConfig.from_pretrained(MODEL_NAME)
with open(f"{ONNX_MODEL}/assets/labels.txt", 'w') as f:
    f.write('\n'.join([v for k, v in sorted(config.id2label.items())]))

!mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets/

In [8]:
!ls -lR {ONNX_MODEL}

onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment:
total 1086452
drwxr-xr-x 2 root root       4096 Jun 17 05:16 assets
-rw-r--r-- 1 root root        847 Jun 17 05:15 config.json
-rw-r--r-- 1 root root 1112503258 Jun 17 05:15 model.onnx
-rw-r--r-- 1 root root        280 Jun 17 05:16 special_tokens_map.json
-rw-r--r-- 1 root root       1231 Jun 17 05:16 tokenizer_config.json

onnx_models/cardiffnlp/twitter-xlm-roberta-base-sentiment/assets:
total 4956
-rw-r--r-- 1 root root      25 Jun 17 05:16 labels.txt
-rw-r--r-- 1 root root 5069051 Jun 17 05:16 sentencepiece.bpe.model


In [9]:
!cat {ONNX_MODEL}/assets/labels.txt

negative
neutral
positive

Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save RoBertaForSequenceClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [10]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 19.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [11]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `XlmRoBertaForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [12]:
from sparknlp.annotator import XlmRoBertaForSequenceClassification

sequenceClassifier = XlmRoBertaForSequenceClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [13]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [14]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your XlmRoBertaForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [15]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 1091560
drwxr-xr-x 3 root root       4096 Jun 17 05:19 fields
drwxr-xr-x 2 root root       4096 Jun 17 05:19 metadata
-rw-r--r-- 1 root root 1112673147 Jun 17 05:20 xlm_roberta_classification_onnx
-rw-r--r-- 1 root root    5069051 Jun 17 05:19 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForSequenceClassification model 😊

In [16]:
sequenceClassifier_loaded = XlmRoBertaForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [17]:
sequenceClassifier_loaded.getClasses()

['neutral', 'positive', 'negative']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [19]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

example = spark.createDataFrame([
    ["I love you!"],
    ["I feel fine to be here."],
    ["I hate you"]
]).toDF("text")

result = pipeline.fit(example).transform(example)
result.select("text", "class.result").show(truncate=False)

+-----------------------+----------+
|text                   |result    |
+-----------------------+----------+
|I love you!            |[positive]|
|I feel fine to be here.|[positive]|
|I hate you             |[negative]|
+-----------------------+----------+



That's it! You can now go wild and use hundreds of `XlmRoBertaForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
